In [1]:
import numpy as np
import pandas
import math
from hmmlearn import hmm
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from create_train_test_val_maps import *

In [60]:
#train_map_revived = open_map('/home/cs231n/data/train')
#val_map_revived = open_map('/home/cs231n/data/val')
#test_map_revived = open_map('/home/cs231n/data/test')


val_map_deser_ok = open_ok_map('/home/cs231n/data/valOK')
train_map_deser_ok = open_ok_map('/home/cs231n/data/trainOK')
test_map_deser_ok = open_ok_map('/home/cs231n/data/testOK')
val_map_deser = open_map('/home/cs231n/data/val')
train_map_deser = open_map('/home/cs231n/data/train')
test_map_deser = open_map('/home/cs231n/data/test')
val_combined = {**val_map_deser, **val_map_deser_ok}
train_combined = {**train_map_deser, **train_map_deser_ok}
test_combined = {**test_map_deser, **test_map_deser_ok}

pc_vec = pandas.read_pickle('pc_vec.pkl')
pc_vec2 = np.array(pc_vec['prob']).reshape((7,1))

cost_prob = np.genfromtxt('cost_prob.csv',delimiter=',')
cost_prob = cost_prob.T


selected_codes = [0,45021, 44004, 43004, 45008, 45002, 45007] # first code is the ok
selected_codes = list(np.sort(selected_codes))
num_codes = len(selected_codes)

In [114]:
selected_repairs = pandas.read_pickle('cleaned_selected_repairs.pkl')
veh_repair_cost_map = {}

for idx, row in selected_repairs.iterrows():
    veh_id = row['Chassis\nReference\nNumber']
    if veh_id not in veh_repair_cost_map:
        veh_repair_cost_map[veh_id] = {}
    cost_str = row['Repair Cost']
    cost = 0
    if cost_str == 'very low':
        cost = 1
    elif cost_str == 'low':
        cost = 2
    elif cost_str == 'medium':
        cost = 3
    elif cost_str == 'high ':
        cost = 4
    elif cost_str == 'very high':
        cost = 5
    if cost==0:
        print("error: {}".format(veh_id))
    veh_repair_cost_map[veh_id][row['ATA6']] = cost

In [61]:
cost_prob.shape

(7, 6)

In [14]:
selected_repairs = pandas.read_pickle('cleaned_selected_repairs.pkl')
veh_repair_map = {}
for veh_id, repairs in selected_repairs.groupby(['Chassis\nReference\nNumber']):
    veh_repair_map[veh_id] = repairs['ATA6'].values

In [15]:
# get global index from (row, col) index
def sub2ind(array_shape, row, col):
    ind = row*array_shape[1] + col
    if row < 0 or row >= array_shape[0]:
        ind = -1
    if col < 0 or col >= array_shape[1]:
        ind = -1
    return ind

# get (row, col) index from global index
def ind2sub(array_shape, ind):
    row = int(ind) / array_shape[1]
    col = ind % array_shape[1]
    if ind < 0:
        row = -1
        col = -1
    if ind >=  array_shape[0]*array_shape[1]:
        row = -1
        col = -1
    return (row, col)

def softmax(x):
    """
    Compute softmax function for input. 
    Use tricks from previous assignment to avoid overflow
    """
    ### YOUR CODE HERE
    xshift = np.max(x, axis = 1)
    xshift = xshift.reshape((x.shape[0],1))
    x = x - xshift
    s = np.exp(x) / np.sum(np.exp(x),axis = 1).reshape((x.shape[0],1))
    ### END YOUR CODE
    return s

In [16]:
def start_train(selected_codes, train_map_revived, num_iter, num_states):
    models = {}
    codes = []
    code_window_map = {}
    for vehicleID in train_map_revived.keys():
        for ATA6code in train_map_revived[vehicleID].keys():
            if ATA6code not in selected_codes:
                continue
            if ATA6code not in code_window_map:
                code_window_map[ATA6code] = {}
            for time_window in train_map_revived[vehicleID][ATA6code].keys():
                if time_window not in code_window_map[ATA6code]:
                    code_window_map[ATA6code][time_window] = []
                for sequence_of_snapshots in train_map_revived[vehicleID][ATA6code][time_window]:
                    code_window_map[ATA6code][time_window].append(sequence_of_snapshots)

    for ATA6code in code_window_map.keys():
        for time_window in code_window_map[ATA6code].keys():
            lengths = []
            listofsequences = code_window_map[ATA6code][time_window]
            X = pandas.concat(listofsequences).as_matrix()
            for sequence in listofsequences:
                lengths.append(sequence.shape[0]) 
            models[(ATA6code, time_window)] = hmm.GaussianHMM(n_components=num_states, n_iter=num_iter).fit(X.astype(float),lengths)
    return models

In [17]:
# TRAIN
num_states = 2
num_time_windows = 10
num_iter = 100
models = start_train(selected_codes, train_combined, num_iter, num_states)

In [123]:
def get_results(alpha,selected_codes,val_map_revived,veh_repair_map,num_time_windows, code_probs, cost_prob, vehicle_repair_cost_map):    
    # VALIDATION ACCURACY
    num_codes = len(selected_codes)
    lengths = []
    for vehicleID in val_map_revived.keys():
        for ATA6code in val_map_revived[vehicleID].keys():
            if ATA6code not in selected_codes:
                continue
            for time_window in val_map_revived[vehicleID][ATA6code].keys():
                lengths.append(len(val_map_revived[vehicleID][ATA6code][time_window]))
    num_val = sum(lengths)
    #print(num_val)

    sample = 0
    labels = []
    
    # mse scores
    code_scores_mse = np.ones(num_val)
    window_scores_mse = np.ones(num_val)
    cost_scores_mse = np.ones(num_val)
    
    # accuracy scores
    code_scores_acc = np.zeros(num_val)
    window_scores_acc = np.zeros(num_val)
    cost_scores_acc = np.zeros(num_val)
    
    log_likelihoods = np.zeros((num_val, num_codes, num_time_windows))
    probabilities = np.zeros((num_val, num_codes, num_time_windows))
    for vehicleID in val_map_revived.keys():
        for ATA6code in val_map_revived[vehicleID].keys():
            if ATA6code not in selected_codes:
                continue
            for time_window in val_map_revived[vehicleID][ATA6code].keys():
                for sequence_of_snapshots in val_map_revived[vehicleID][ATA6code][time_window]:
                    labels.append([selected_codes.index(ATA6code),time_window])
                    x = sequence_of_snapshots.as_matrix() 
                    for pair in models.keys():
                        c = selected_codes.index(pair[0])
                        w = pair[1]
                        if c == 0 & w == 0:
            
                            #continue
                            #print(x.shape)
                            temp_score = models[pair].score(x.astype(float))
                            for i in range(len(log_likelihoods[sample,c,:])):
                                log_likelihoods[sample,c,i] = temp_score
                        else:
                            #print(x.shape)
                            log_likelihoods[sample,c,w] = models[pair].score(x.astype(float))
                    
                    # Compute Softmax Prob Matrix
                    probabilities_temp = softmax(log_likelihoods[sample].reshape(1,num_codes*num_time_windows))
                    probabilities[sample] = probabilities_temp.reshape(num_codes, num_time_windows) 
                    posterior_code_probs = np.sum(probabilities[sample]*code_probs,axis=1).reshape(1,7)
                    #print(posterior_code_probs.shape)
                    #print(cost_prob.shape)
                    expected_cost = np.dot(np.dot(posterior_code_probs,cost_prob),np.arange(6).reshape(6,1))
                    # Get marginal max 
                    argmax_window = np.argmax(np.dot(probabilities[sample].T,code_probs))
                    argmax_code = np.argmax(np.sum(probabilities[sample]*code_probs,axis=1))
                    #max_index = np.unravel_index(probabilities[sample].argmax(), probabilities[sample].shape)

                    # MSE
                    #----------
                    # code
                    if  argmax_code == selected_codes.index(ATA6code):
                        code_scores_mse[sample] = 0
                    elif vehicleID in veh_repair_map and selected_codes[argmax_code] in veh_repair_map[vehicleID]:
                        code_scores_mse[sample] = 1-alpha
                    # window
                    window_scores_mse[sample] = math.pow(argmax_window - time_window,2)
                    # cost
                    if vehicleID in vehicle_repair_cost_map and ATA6code in vehicle_repair_cost_map[vehicleID]:
                        cost_scores_mse[sample] = math.pow(expected_cost - vehicle_repair_cost_map[vehicleID][ATA6code],2)
                    else:
                        cost_scores_mse[sample] = math.pow(expected_cost - 0,2)
                    
                    # Accuracy
                    #----------
                    # code
                    #print('argmax_code: ', argmax_code)
                    #print('vehicleID: ', vehicleID)
                    if  argmax_code == selected_codes.index(ATA6code):
                        code_scores_acc[sample] = 1
                    elif vehicleID in veh_repair_map and selected_codes[argmax_code] in veh_repair_map[vehicleID]:
                        code_scores_acc[sample] = alpha
                    # window
                    window_scores_acc[sample] = 1.0/(1.0 + math.pow(argmax_window - time_window,2))
                    # cost
                    if vehicleID in vehicle_repair_cost_map and ATA6code in vehicle_repair_cost_map[vehicleID]:
                        cost_scores_acc[sample] = 1.0/(1.0 + math.pow(expected_cost - vehicle_repair_cost_map[vehicleID][ATA6code],2))
                    else:
                        cost_scores_acc[sample] = 1.0/(1.0 + math.pow(expected_cost - 0,2))
                    sample = sample+1        
    
    #print(code_scores_acc)
    labels = np.asarray(labels)
    code_error = np.mean(code_scores_mse)
    window_error = np.mean(window_scores_mse) 
    cost_error = np.mean(cost_scores_mse)
    code_accuracy = np.mean(code_scores_acc)
    window_accuracy = np.mean(window_scores_acc)
    cost_accuracy = np.mean(cost_scores_acc)
    
    return code_error, window_error, cost_error, code_accuracy, window_accuracy, cost_accuracy

In [124]:
# VALIDATE
alpha = 0.5
code_probs = np.ones((num_codes,1))
code_error, window_error, cost_error, code_accuracy, window_accuracy, cost_accuracy = get_results(alpha,selected_codes,test_combined,veh_repair_map,num_time_windows, pc_vec2, cost_prob, veh_repair_cost_map)
print('ATA6 Code Error: ', code_error)
print('ATA6 Code Accuracy: ', code_accuracy)
print('Window Error: ', window_error)
print('Window Accuracy: ', window_accuracy)
print('Cost Error: ', cost_error)
print('Cost Accuracy: ', cost_accuracy)

ATA6 Code Error:  0.564207650273
ATA6 Code Accuracy:  0.435792349727
Window Error:  21.2759562842
Window Accuracy:  0.363338268133
Cost Error:  1.4055536304
Cost Accuracy:  0.766069413941
